In [1]:
import numpy as np
import networkx as nx
import matplotlib.pyplot as plt
import functools
import random
import time
import random
from pygame_visualization import VRPVisualizator

pygame 2.0.1 (SDL 2.0.14, Python 3.8.10)
Hello from the pygame community. https://www.pygame.org/contribute.html


In [2]:
vizu = VRPVisualizator()

In [3]:
def manhattan(a, b):
    return sum(abs(val1-val2) for val1, val2 in zip(a,b))

In [4]:
def generate_coordinates(n_nodes: int, x_coords_range, y_coords_range, min_distance: float):
    generated_coords = [(0, 0)]
    vizu.add_node(0, 0)
    for i in range(n_nodes - 1):
        is_ok = False
        while not is_ok:
            coord = (random.uniform(*x_coords_range), random.uniform(*y_coords_range))
            is_ok = True
            for node in generated_coords:
                if manhattan(coord, node) < min_distance:
                    is_ok = False
        generated_coords.append(coord)
        vizu.add_node(coord[0], coord[1])
    return generated_coords

In [5]:
def generate_graph_matrix(length: int):
    coords = generate_coordinates(length, (-600, 600), (-400, 400), 200)
    matrix = np.empty((length, length))
    matrix[:] = np.nan
    for i in range(length):
        for j in range(length):
            if i != j:
                dist = manhattan(coords[i], coords[j])
                matrix[i][j] = dist
                matrix[j][i] = dist
    return matrix

In [6]:
vizu.clear_nodes()
graph = generate_graph_matrix(20)

In [7]:
vizu.set_scaling(1.2)

In [8]:
print(graph)

[[          nan  624.48485162  225.2099674   538.08055285  610.75980836
   653.68039877  324.70861896  591.2009973   303.96175938  701.70504239
   231.98411139  549.32909696  378.56449378  814.48225856  318.80733709
   791.82041467  577.05336927  566.6488611   854.83570971  786.02805728]
 [ 624.48485162           nan  849.69481901  797.68864385  731.37597021
   421.66145258  308.01222358 1215.68584891  320.52309223 1030.21573168
   639.76405455 1173.81394858  437.11438864  693.8660967   943.29218871
  1416.30526629  456.43720742  209.73512101  230.3508581   906.64421913]
 [ 225.2099674   849.69481901           nan  482.62689015  555.30614565
   878.89036617  541.68259544  365.9910299   529.17172678  646.25137969
   209.93076446  324.11912957  603.77446118  869.93592126  324.50746711
   566.61044727  637.97562766  791.8588285  1080.04567711  730.57439458]
 [ 538.08055285  797.68864385  482.62689015           nan  222.97213512
  1080.35816729  862.78917181  464.89815878  477.16555162  23

In [17]:
#Impact of pheromones
ALPHA = 0.6
#Impact of weights
BETA = 0.5

class Ant():
    def __init__(self, init_pos: int, vizualizator: VRPVisualizator) -> None:
        self.init_pos = init_pos
        self.vizualizator = vizualizator
        self.current_pos = init_pos
        self.path_history = [init_pos]
        self.can_continue = True

    def add_to_path_history(self, node) -> None:
        self.path_history.append(node)
        #self.vizualizator.set_path(self.path_history)

    def move(self, dest: list, pheromones: list) -> None:
        probabilities = []
        denominator = 0
        
        #Calculate the denominator first
        for i in range(len(dest)):
            if not np.isnan(dest[i]) and i not in self.path_history:
                denominator += pheromones[i]**ALPHA * (1/dest[i])**BETA

        #Calculate probabilities of picking one path
        for node, length in enumerate(dest):
            if node in self.path_history:
                probabilities.append(0)
            elif not np.isnan(dest[node]):
                nominator = pheromones[node]**ALPHA * (1/dest[node])**BETA
                probabilities.append(nominator / denominator)
            else:
                probabilities.append(np.nan)
        
        #If there is no path available return false
        if np.nansum(probabilities) == 0:
            self.add_to_path_history(self.init_pos)
            self.can_continue = False
            return

        #Roulette wheel
        cumulative_sum = []
        for i in range(len(probabilities)):
            if np.isnan(probabilities[i]):
                cumulative_sum.append(np.nan)
            elif np.nansum(cumulative_sum) == 0:
                cumulative_sum.append(1.0)
            else:
                cumulative_sum.append(np.nansum(probabilities[i:len(probabilities)]))
        
        #Pick a destination
        rand = random.uniform(0, 1)
        dest_picked = None
        cumulative_sum.append(0)

        for i in range(0, len(cumulative_sum)-1):
            p = cumulative_sum[i]
            nextp = cumulative_sum[i+1] if not np.isnan(cumulative_sum[i+1]) else cumulative_sum[i+2] if not np.isnan(cumulative_sum[i+2]) else cumulative_sum[i+3]
        
            if not np.isnan(p) and rand <= p and rand >= nextp:
                dest_picked = i

        self.current_pos = dest_picked
        self.add_to_path_history(self.current_pos)

In [18]:
RHO = 0.8
Q = 1

class ACO():
    def __init__(self, graph, start: int, vizualizator: VRPVisualizator) -> None:
        self.graph = graph
        self.start = start
        self.vizualizator = vizualizator
        self.all_paths = []
        self.pheromones = np.ones(graph.shape)
        #self.vizualizator.set_pheromones(self.pheromones)

    def run(self, iter: int) -> list:
        for i in range(iter):
            paths = self.launch_ants(40)
            for path in paths:
                self.all_paths.append(path)
            self.update_pheromones(paths)
        final_path = self.get_final_path(self.all_paths)
        self.vizualizator.set_path(final_path)
        return final_path

    def launch_ants(self, amount: int) -> list:
        paths = []
        ants = [Ant(self.start, self.vizualizator) for i in range(amount)]
        for ant in ants:
            while ant.can_continue:
                ant.move(self.graph[ant.current_pos], self.pheromones[ant.current_pos])
            paths.append(ant.path_history)
        return paths

    def update_pheromones(self, paths: list) -> None:
        #Evaporation
        self.pheromones *= 1 - RHO
        #New pheromones
        for path in paths:
            total_distance = self.calc_total_distance(path)
            for i in range(len(path)-1):
                cur_path = path[i]
                next_path = path[i+1]
                self.pheromones[cur_path][next_path] += Q / total_distance
                self.pheromones[next_path][cur_path] += Q / total_distance
        #self.vizualizator.set_pheromones(self.pheromones)

    def calc_total_distance(self, full_path: list) -> float:
        distance = 0
        for i in range(len(full_path)-1):
            cur_path = full_path[i]
            next_path = full_path[i+1]
            distance += self.graph[cur_path][next_path]
        return distance

    def get_final_path(self, paths: list) -> list:
        best_path = paths[0]
        best_i = 0
        lowest_distance = self.calc_total_distance(paths[0])
        for i, p in enumerate(paths):
            distance = self.calc_total_distance(p)
            if distance <= lowest_distance:
                best_path = p
                lowest_distance = distance
                best_i = i
        print(lowest_distance)
        print(best_i)
        print(len(paths))
        print('count:', paths.count(best_path) + paths.count(reversed(best_path)))
        return best_path


In [19]:
aco = ACO(graph, 0, vizu)
start = time.time()
path = aco.run(80)
print(path)
print(time.time()-start)

7384.71702314709
665
3200
count: 1
[0, 8, 17, 1, 18, 5, 6, 16, 13, 14, 7, 4, 3, 19, 9, 11, 2, 15, 10, 12, 0]
82.43549942970276
